<a href="https://colab.research.google.com/github/AyaElsawyElghaysh/RecommenderSystem/blob/main/recommender_sys_movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective :


1.   Bulid simple recommender system based on item popularity
2.   Build Content based filtering based on user preferences and metadata

3.  Bulid conllaborative filtering based on users similarity and some user`s preferences






In [1]:
# Imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_kernels
import matplotlib.pyplot as plt
from pathlib import Path
import os
import re
import html
import string
import unicodedata
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer


from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import regularizers
from tensorflow.keras import metrics
from tensorflow.keras.utils import plot_model
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences





[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#Load dataset


In [2]:
#idbm 
data=pd.read_csv("/content/drive/MyDrive/RecommenderSys dataset/movies_metadata.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
data.shape

(45466, 24)

In [5]:
data_new=data.iloc[:25000,:] #to handel resourses

In [6]:
data_new=pd.DataFrame(data_new)
data_new.shape

(25000, 24)

## Excercise

Build Simple Recommender System based on the metric below:
- weightedRating(WR) = ((v/v+m).R) + ((m/v+m).C)
 * v >> is the number of votes for the movie. (vote_count)
 * m >> is the min votes required to be listed in chart. (based on negative vote)
 * R >> is the average rating of the movie. (vote_average)
 * C >> is the mean vote across the the whole report. (calculate from data)




In [7]:
data_simple_R=data_new[['title','vote_average','vote_count']]

In [8]:
data_simple_R

,title,vote_average,vote_count
0,Toy Story,7.7,5415.0
1,Jumanji,6.9,2413.0
2,Grumpier Old Men,6.5,92.0
3,Waiting to Exhale,6.1,34.0
4,Father of the Bride Part II,5.7,173.0
...,...,...,...
24995,Deranged,4.1,7.0
24996,When the Game Stands Tall,6.3,134.0
24997,Aashiqui 2,6.8,41.0
24998,ABCs of Death 2,5.5,89.0


In [9]:
# calculate C
C = data_simple_R['vote_average'].mean()
print(C)

5.8926434114729185


In [10]:
# Calculate M which is the min votes reqired to be listed in our recommendation 
M = data_simple_R['vote_count'].quantile(0.90)
print(M)
print(data_simple_R.shape)

312.2999999999993
(25000, 3)


In [11]:
#filter data based on the above ratio
filter_data=data_simple_R.copy().loc[data_simple_R['vote_count']>=160]

In [12]:
filter_data

,title,vote_average,vote_count
0,Toy Story,7.7,5415.0
1,Jumanji,6.9,2413.0
4,Father of the Bride Part II,5.7,173.0
5,Heat,7.7,1886.0
8,Sudden Death,5.5,174.0
...,...,...,...
24900,Trash,7.1,243.0
24902,Wild Tales,7.7,878.0
24939,Dumb and Dumber To,5.4,1140.0
24959,Dear White People,6.2,222.0


#computer function weigthed_rating

In [13]:
# Function that computes the weighted rating of each movie
def weighted_rating(x, M=M, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+M) * R) + (M/(M+v) * C)

In [14]:
#define score column 
filter_data["score"]=filter_data.apply(weighted_rating,axis=1)

In [15]:
filter_data

,title,vote_average,vote_count,score
0,Toy Story,7.7,5415.0,7.601448
1,Jumanji,6.9,2413.0,6.784564
4,Father of the Bride Part II,5.7,173.0,5.823970
5,Heat,7.7,1886.0,7.443239
8,Sudden Death,5.5,174.0,5.752154
...,...,...,...,...
24900,Trash,7.1,243.0,6.420984
24902,Wild Tales,7.7,878.0,7.225802
24939,Dumb and Dumber To,5.4,1140.0,5.505937
24959,Dear White People,6.2,222.0,6.020349


In [16]:
#Sort movies based on score calculated above
filter_data = filter_data.sort_values('score',ascending=False)
#Print the top 15 movies
filter_data[['title', 'vote_count', 'vote_average', 'score']].head(15)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.406084
834,The Godfather,6024.0,8.5,8.371490
12481,The Dark Knight,12269.0,8.3,8.240243
2843,Fight Club,9678.0,8.3,8.224745
292,Pulp Fiction,8670.0,8.3,8.216300
522,Schindler's List,4436.0,8.3,8.141666
23673,Whiplash,4376.0,8.3,8.139640
5481,Spirited Away,3968.0,8.3,8.124354
351,Forrest Gump,8147.0,8.2,8.114817
2211,Life Is Beautiful,3643.0,8.3,8.109922


In [17]:
#we arrived to the end goal and bulit a simple recommender sys based on item popularity

#Now i will build Content based filter

In [18]:
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [19]:
over_view_preprocess= data_new['overview']

In [20]:
over_view_preprocess.isnull().sum()

209

In [21]:
data_new['overview']=data_new['overview'].fillna('')

In [22]:
over_view_preprocess.isnull().sum()

0

In [23]:
data_new['overview'].iloc[0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."

#Text Preprocess

In [24]:
tf_IDF=TfidfVectorizer(stop_words='english')
transormed_review=tf_IDF.fit_transform(data_new['overview'])
transormed_review.shape

(25000, 53130)

In [25]:
tf_IDF.transform([data_new['overview'].iloc[100]]).toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [26]:
#after we converted text to tfidf we will compute similarity between all reviews
#take first 10000 becuase of resources limits
sim_matrix=linear_kernel(transormed_review,transormed_review)

In [27]:
#get indecies of out titels 
#Construct a reverse map of indices and movie titles
indices = pd.Series(data_new.index, index=data_new['title']).drop_duplicates()

In [28]:
#sorted(list(enumerate(sim_matrix[idx])),key=lambda x: x[1],reverse=True)

In [29]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=sim_matrix):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return data_new['title'].iloc[movie_indices]

In [30]:
get_recommendations("The Godfather")

1178      The Godfather: Part II
1914     The Godfather: Part III
23126                 Blood Ties
11297           Household Saints
10821                   Election
17729          Short Sharp Shock
8653                Violent City
13177               I Am the Law
6977             Queen of Hearts
6711                    Mobsters
Name: title, dtype: object

In [31]:
get_recommendations('The Dark Knight Rises') #great!!

12481                                      The Dark Knight
150                                         Batman Forever
1328                                        Batman Returns
15511                           Batman: Under the Red Hood
585                                                 Batman
21194    Batman Unmasked: The Psychology of the Dark Kn...
9230                    Batman Beyond: Return of the Joker
18035                                     Batman: Year One
19792              Batman: The Dark Knight Returns, Part 1
3095                          Batman: Mask of the Phantasm
Name: title, dtype: object

## Excercise

In this excercise, we want to recommend movies based on some extra metadata _features_, like `'cast', 'crew', 'keywords', 'genres'`. One important difference is that you use the `CountVectorizer()` instead of TF-IDF. This is because you do not want to down-weight the presence of an actor/director if he or she has acted or directed in relatively more movies. It doesn't make much intuitive sense.

You are asked to recommend movies on the new cosine_similarity, pair wise `CountVectorizer()` scores.

In [7]:
credits = pd.read_csv("/content/drive/MyDrive/RecommenderSys dataset/credits.csv")
keywords = pd.read_csv("/content/drive/MyDrive/RecommenderSys dataset/keywords.csv")

credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [8]:
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [9]:
# Remove rows with bad IDs.
data_new = data_new.drop([19730])

# Convert IDs to int. Required for merging
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
data_new['id'] = data_new['id'].astype('int')
#merge two tabels with origin table
data_new = data_new.merge(credits, on='id')
data_new = data_new.merge(keywords, on='id')

In [10]:
data_new.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew', 'keywords'],
      dtype='object')

In [11]:
data_new['crew'].head()

0    [{'credit_id': '52fe4284c3a36847f8024f49', 'de...
1    [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...
2    [{'credit_id': '52fe466a9251416c75077a89', 'de...
3    [{'credit_id': '52fe44779251416c91011acb', 'de...
4    [{'credit_id': '52fe44959251416c75039ed7', 'de...
Name: crew, dtype: object

In [12]:
data_new['cast'].head()

0    [{'cast_id': 14, 'character': 'Woody (voice)',...
1    [{'cast_id': 1, 'character': 'Alan Parrish', '...
2    [{'cast_id': 2, 'character': 'Max Goldman', 'c...
3    [{'cast_id': 1, 'character': "Savannah 'Vannah...
4    [{'cast_id': 1, 'character': 'George Banks', '...
Name: cast, dtype: object

In [13]:
data_new['keywords'].head()

0    [{'id': 931, 'name': 'jealousy'}, {'id': 4290,...
1    [{'id': 10090, 'name': 'board game'}, {'id': 1...
2    [{'id': 1495, 'name': 'fishing'}, {'id': 12392...
3    [{'id': 818, 'name': 'based on novel'}, {'id':...
4    [{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...
Name: keywords, dtype: object

In [14]:
# Parse the stringified features into their corresponding python objects
from ast import literal_eval

features =['cast', 'crew', 'keywords', 'genres']
for feature in features:
    data_new[feature]= data_new[feature].apply(literal_eval)

In [15]:
# data_new['crew'].values

In [16]:
# Get the director's name from the crew feature. If director is not listed, return NaN
def get_director(x):
    for i in x:
        if i['job'] =='Director':
            return i['name']
    return np.nan

In [17]:
# Returns the list top 3 elements or entire list; whichever is more.
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [18]:
x=[{'id': 931, 'name': 'jealousy'}]
get_list(x)

['jealousy']

In [19]:
# Define new director, cast, genres and keywords features that are in a suitable form.
data_new['director'] = data_new['crew'].apply(get_director)

In [20]:
data_new['director'] 

0            John Lasseter
1             Joe Johnston
2            Howard Deutch
3          Forest Whitaker
4            Charles Shyer
               ...        
25146           Neil Jones
25147        Thomas Carter
25148           Mohit Suri
25149        Rodney Ascher
25150    Oliver Hockenhull
Name: director, Length: 25151, dtype: object

In [21]:
features = ['cast', 'keywords', 'genres']
for feature in features:
    data_new[feature] = data_new[feature].apply(get_list)

In [22]:
data_new['keywords'] 

0                                     [jealousy, toy, boy]
1        [board game, disappearance, based on children'...
2             [fishing, best friend, duringcreditsstinger]
3        [based on novel, interracial relationship, sin...
4                       [baby, midlife crisis, confidence]
                               ...                        
25146                                                   []
25147       [american football, sport, high school sports]
25148                       [artist, musical, tragic love]
25149              [anthology, woman director, mumblegore]
25150                        [medicine, war on drugs, lsd]
Name: keywords, Length: 25151, dtype: object

In [23]:
# Print the new features of the first 3 films
data_new[['title', 'cast', 'director', 'keywords', 'genres']].head(10)

,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[Animation, Comedy, Family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[Romance, Comedy]"
3,Waiting to Exhale,"[Whitney Houston, Angela Bassett, Loretta Devine]",Forest Whitaker,"[based on novel, interracial relationship, sin...","[Comedy, Drama, Romance]"
4,Father of the Bride Part II,"[Steve Martin, Diane Keaton, Martin Short]",Charles Shyer,"[baby, midlife crisis, confidence]",[Comedy]
5,Heat,"[Al Pacino, Robert De Niro, Val Kilmer]",Michael Mann,"[robbery, detective, bank]","[Action, Crime, Drama]"
6,Sabrina,"[Harrison Ford, Julia Ormond, Greg Kinnear]",Sydney Pollack,"[paris, brother brother relationship, chauffeur]","[Comedy, Romance]"
7,Tom and Huck,"[Jonathan Taylor Thomas, Brad Renfro, Rachael ...",Peter Hewitt,[],"[Action, Adventure, Drama]"
8,Sudden Death,"[Jean-Claude Van Damme, Powers Boothe, Dorian ...",Peter Hyams,"[terrorist, hostage, explosive]","[Action, Adventure, Thriller]"
9,GoldenEye,"[Pierce Brosnan, Sean Bean, Izabella Scorupco]",Martin Campbell,"[cuba, falsely accused, secret identity]","[Adventure, Action, Thriller]"


In [24]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [25]:
features = ['cast', 'keywords', 'director', 'genres']
for feature in features:
    data_new[feature] = data_new[feature].apply(clean_data)

In [26]:
data_new[features]

,cast,keywords,director,genres
0,"[tomhanks, timallen, donrickles]","[jealousy, toy, boy]",johnlasseter,"[animation, comedy, family]"
1,"[robinwilliams, jonathanhyde, kirstendunst]","[boardgame, disappearance, basedonchildren'sbook]",joejohnston,"[adventure, fantasy, family]"
2,"[waltermatthau, jacklemmon, ann-margret]","[fishing, bestfriend, duringcreditsstinger]",howarddeutch,"[romance, comedy]"
3,"[whitneyhouston, angelabassett, lorettadevine]","[basedonnovel, interracialrelationship, single...",forestwhitaker,"[comedy, drama, romance]"
4,"[stevemartin, dianekeaton, martinshort]","[baby, midlifecrisis, confidence]",charlesshyer,[comedy]
...,...,...,...,...
25146,"[marciadovales, craigfairbrass, victoriabroom]",[],neiljones,[horror]
25147,"[jimcaviezel, michaelchiklis, lauradern]","[americanfootball, sport, highschoolsports]",thomascarter,[drama]
25148,"[adityaroykapoor, shraddhakapoor, shaadrandhawa]","[artist, musical, tragiclove]",mohitsuri,"[drama, romance]"
25149,"[ericjacobus, julianbarratt, ianvirgo]","[anthology, womandirector, mumblegore]",rodneyascher,[horror]


In [27]:
def create_soup(x):  #deal all features as one feature to be easy representation
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

In [28]:
# Create a new soup feature
data_new['soup'] = data_new.apply(create_soup, axis=1)

In [29]:
data_new['soup'].head(5).values

array(['jealousy toy boy tomhanks timallen donrickles johnlasseter animation comedy family',
       "boardgame disappearance basedonchildren'sbook robinwilliams jonathanhyde kirstendunst joejohnston adventure fantasy family",
       'fishing bestfriend duringcreditsstinger waltermatthau jacklemmon ann-margret howarddeutch romance comedy',
       'basedonnovel interracialrelationship singlemother whitneyhouston angelabassett lorettadevine forestwhitaker comedy drama romance',
       'baby midlifecrisis confidence stevemartin dianekeaton martinshort charlesshyer comedy'],
      dtype=object)

In [30]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(data_new['soup'])
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
# Reset index of your main DataFrame and construct reverse mapping as before
data_new = data_new.reset_index()
indices = pd.Series(data_new.index, index=data_new['title'])

In [31]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim2):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return data_new['title'].iloc[movie_indices]

In [32]:
get_recommendations("Father of the Bride Part II")

3977                                             Baby Boom
6842                                   Father of the Bride
2381                                        ¡Three Amigos!
22528                                            The Guide
3204                                            Hanging Up
9217     Das merkwürdige Verhalten geschlechtsreifer Gr...
10079                             Jiminy Glick in Lalawood
7444                                        The Lonely Guy
19651                                              The Lie
23782                                           Mama's Boy
Name: title, dtype: object

In [33]:
get_recommendations("The Godfather")

1930            The Godfather: Part III
1195             The Godfather: Part II
15571                   The Rain People
18894                         Last Exit
7981     The Night of the Following Day
18215                 The Son of No One
7752                           Mitchell
23494               Hide in Plain Sight
1182                     Apocalypse Now
1644                   Ill Gotten Gains
Name: title, dtype: object